In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import json
from collections import defaultdict
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)

In [ ]:
from lib.utilities_alignment import (create_warp_transforms, parse_elastix)
def transform_create_alignment(points, transform):
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

In [ ]:
animal = 'DK55'
downsample_factor = 32
CSV_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}'

In [ ]:
IMG_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
cshl_csvfile = 'cshl.premotor.csv'
cshl_csvpath = os.path.join(CSV_PATH, cshl_csvfile)
cshl_df = pd.read_csv(cshl_csvpath, names=['section','x','y'])

In [ ]:
sections = cshl_df['section'].unique().tolist()
print(sections)

In [ ]:
section_size = {}
for section_num in sections:
    filename = str(section_num).zfill(3) + '.tif'
    filepath = os.path.join(IMG_PATH, 'CH3/full', filename)
    input_image = Image.open(filepath)
    rotated_height = input_image.width
    section_size[section_num] = rotated_height
    input_image.close()

In [ ]:
for index, row in cshl_df.iterrows():
    section_num = int(row['section'])
    rotated_height = section_size[section_num]
    cshl_df.at[index,'xp'] = cshl_df.at[index,'y'] 
    cshl_df.at[index,'yp'] = rotated_height - cshl_df.at[index,'x'] 

In [ ]:
cshl_df.head()

In [ ]:
# Test on unaligned, normalized and rotated images. These turn out nicely
section_test_num = 228
filename = f'{str(section_test_num).zfill(3)}.tif'
filepath = os.path.join(IMG_PATH, 'CH3/normalized', filename)
img = cv2.imread(filepath, -1)
radius = 5
color = (0,255,1)
df = cshl_df.loc[cshl_df['section'] == section_test_num]
for index, row in df.iterrows():
    x = round(row['xp']/downsample_factor) 
    y = round(row['yp']/downsample_factor)
    #print(x,y)
    cv2.circle(img, (int(x), int(y)), radius, color, 2)

outpath = os.path.join(IMG_PATH,'CH3',f'{section_test_num}.out.tif')
cv2.imwrite(outpath, img)

## Get the annotation points

In [ ]:
section_vertices = defaultdict(list)
for index, row in cshl_df.iterrows():
    section_num = int(row['section'])
    x = row['xp']
    y = row['yp']
    section_vertices[section_num].append([x,y])

## Reproduce create_clean transform

In [ ]:
# find the difference between the image stack size which is consistent for all images
# and the individual shape of the aligned image: use thumbnail 
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH3/thumbnail'
section_offset = {}

downsample_factor = 32
# original image is long and narrow with the brain stem at the bottom
# work with downsampled sizes
fixed_width = 1875
fixed_height = 1062
#1875x1062
# below is orig
#aligned_shape = np.array((width, height)) / downsample_factor
#below works
cleaned_section_points = defaultdict(list)

for section_num in sections:
    filename = str(section_num).zfill(3) + '.tif'
    filepath = os.path.join(INPUT, filename)    
    input_image = Image.open(filepath)
    xshift = (fixed_width - input_image.height) / 2 
    yshift = (fixed_height - input_image.width) / 2
    vertices = section_vertices[section_num]
    points = np.array(vertices) / downsample_factor
    shifts = np.array([xshift, yshift])
    print(filename, shifts)
    cleaned_section_points[section_num] = points + shifts


Test create_clean transform

In [ ]:
# test on image that has been rotated and placed in standard size
filename = f'{str(section_test_num).zfill(3)}.tif'
filepath = os.path.join(IMG_PATH, 'CH3/thumbnail_cleaned_normalized', filename)
img = cv2.imread(filepath, -1)
points = cleaned_section_points[section_test_num]
for point in points:
    x = point[0]
    y = point[1]
    #print(x,y)
    cv2.circle(img, (int(x), int(y)), radius, color, 2)
outpath = os.path.join(IMG_PATH,'CH3',f'{section_test_num}.cleaned.out.tif')
cv2.imwrite(outpath, img)
fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Placed section:{}'.format(section_test_num), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()

## Reproduce create_alignment transform

In [ ]:
transforms = parse_elastix(animal)
warp_transforms = create_warp_transforms(animal, transforms, downsample=True)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for filename, transform in ordered_transforms:
    section_num = int(filename.split('.')[0])
    transform = np.linalg.inv(transform) 
    section_transform[section_num] = transform

## Alignment of annotation coordinates

In [ ]:
cleaned_section_points = defaultdict(list)
aligned_section_points = defaultdict(list)
for section_num, vertices in section_vertices.items():
    points = np.array(vertices) / downsample_factor
    cleaned_points = points + section_offset[section_num] # create_clean offset
    cleaned_section_points[section_num] = cleaned_points
    points = transform_create_alignment(cleaned_points, section_transform[section_num]) # create_alignment transform
    aligned_section_points[section_num] = [points]
    

In [ ]:
points = cleaned_section_points[section_test_num]
print(type(points), section_test_num)

In [ ]:
data = []
for section,v in aligned_section.items():
    for x,y in v[0]:
        data.append([x,y,section])    

df = pd.DataFrame(data, columns=['x','y','section'])
df = df.astype({'section':'int32','x': 'float64', 'y':'float64'})
outfile = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/cshl2dk.aligned.csv'
#df.to_csv(outfile, index=False, header=False)
df.head()

To this point, aligned_section_structure_polygons variable contains the aligned polygon vertices for each structure in each section. 
From now on, we introduce how to draw these points to numpy array or neuroglancer

In [ ]:
sections

In [ ]:
section = 222
filename = f'{str(section).zfill(3)}.tif'
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK55/preps/CH3'
outpath = os.path.join(INPUT, f'{section}.out.tif')
filepath = os.path.join(INPUT, 'thumbnail_aligned_norm', filename)
#filepath = os.path.join(INPUT, '224.norm.rotated.tif')
img = cv2.imread(filepath, -1)
#img = (img/256).astype(np.uint8)
#clahe = cv2.createCLAHE(clipLimit=30.0, tileGridSize=(4, 4))
#img = clahe.apply(img)

radius = 5
color = (0,255,1)
#df = cshl_df.copy()
df = df.loc[df['section'] == section]
for index, row in df.iterrows():
    x = round(row['x']) 
    y = round(row['y'])
    print(x,y)
    cv2.circle(img, (int(x), int(y)), radius, color, 2)

cv2.imwrite(outpath, img)
fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Aligned section:{}'.format(section), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()